In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh


In [36]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


### Load example data
Datasets used in Finetuning.py example of the GitHub of GEITje. 

In [37]:

from datasets import DatasetDict, load_dataset, concatenate_datasets


no_robots_nl = load_dataset('Rijgersberg/no_robots_nl')
print(no_robots_nl)
no_robots_nl["train_sft"]=no_robots_nl["train_sft"].select(range(2))
no_robots_nl["test_sft"]=no_robots_nl["test_sft"].select(range(2))

ultrachat_nl = load_dataset('Rijgersberg/ultrachat_10k_nl')
print(ultrachat_nl)
ultrachat_nl["train_sft"]=ultrachat_nl["train_sft"].select(range(2))
ultrachat_nl["test_sft"]=ultrachat_nl["test_sft"].select(range(2))

chat_dataset = DatasetDict({
    'train_sft': concatenate_datasets([no_robots_nl['train_sft'],
                                        ultrachat_nl['train_sft']]).shuffle(seed=42),
    'test_sft': concatenate_datasets([no_robots_nl['test_sft'],
                                        ultrachat_nl['test_sft']]).shuffle(seed=42),
})

DatasetDict({
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 500
    })
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 9500
    })
})
DatasetDict({
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'messages_nl'],
        num_rows: 500
    })
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'messages_nl'],
        num_rows: 9500
    })
})


In [38]:
print(chat_dataset['train_sft'][0])
for i in chat_dataset['train_sft'][0]:
    print(i,'\n', chat_dataset['train_sft'][0][i])

{'prompt': 'Write an informative and persuasive article of 1000 words or more that discusses the environmental and economic benefits of using renewable energy sources, such as solar, wind and hydropower, for powering mining operations. Your article should highlight various case studies, statistics and research findings that demonstrate the feasibility, efficiency and cost-effectiveness of renewable energy solutions in the mining industry. Also, provide insight into the potential challenges, barriers and solutions for implementing renewable energy projects in mining sites worldwide. Use credible sources, technical terms and examples to strengthen your arguments and engage your readers. Finally, offer practical recommendations, policy suggestions or call to actions that inspire readers to support and promote the transition towards a cleaner and more sustainable mining industry.', 'prompt_id': 'd89a653d6d671c30ef07c318e46ef44df29f25079b2dd5d21ade1af4db7efe0a', 'messages': [{'content': 'Wr

Inspection: we only need the columns prompt_id (equal to doc id) and messages_nl. 

### Load Amsterdam data

In [39]:
import pandas as pd

df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
# display(df)

### Tokenize text using GEITje/Mistral tokenizer

In [40]:
from transformers import AutoTokenizer

def get_tokens(model_name, df, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_len_tokens = []
    for txt in all_texts:
        tokens = tokenizer.tokenize(txt)
        all_len_tokens.append(tokens)

    df[new_col_name] = all_len_tokens
    return df

In [50]:
from datasets import Dataset

dataset = Dataset.from_pandas(tokens_df.iloc[0:1])
dataset2 =  Dataset.from_pandas(tokens_df.iloc[1:2])


chat = DatasetDict(
    {'train':dataset,
    'test': dataset2})
print(chat)
print(chat_dataset)


DatasetDict({
    train: Dataset({
        features: ['label', 'path', 'id', 'set', 'text', 'tokens', 'token_count', 'clean_tokens', 'clean_tokens_count', 'pdf_path', 'num_pages', 'clean_text', '4split', '2split', 'GEITjeTokens', '__index_level_0__'],
        num_rows: 1
    })
    test: Dataset({
        features: ['label', 'path', 'id', 'set', 'text', 'tokens', 'token_count', 'clean_tokens', 'clean_tokens_count', 'pdf_path', 'num_pages', 'clean_text', '4split', '2split', 'GEITjeTokens', '__index_level_0__'],
        num_rows: 1
    })
})
DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 4
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 4
    })
})


In [61]:
tokens_df = get_tokens('Rijgersberg/GEITje-7B-chat-v2', df.iloc[0:2], 'text', 'GEITjeTokens')
display(tokens_df)

/tmp/ipykernel_31574/3004451162.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col_name] = all_len_tokens


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,4split,2split,GEITjeTokens
12190,Raadsadres,/home/azureuser/cloudfiles/code/blobfuse/raads...,24894,test,x Gemeente\n\n% Amsterdam\n\n1 Amsterdam.nl\n\...,"[x, Gemeente, %, Amsterdam, 1, Amsterdam.nl, 1...",55,"[Gemeente, Amsterdam, Amsterdam.nl, 1.1, Formu...",31,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Amsterdam.nl 1.1 Formulier ...,train,train,"[▁x, ▁Geme, ente, <0x0A>, <0x0A>, %, ▁Amsterda..."
7210,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,13829,test,Bezoekadres XX Gemeente Amsterdam\nPieter Cala...,"[Bezoekadres, XX, Gemeente, Amsterdam, Pieter,...",515,"[Bezoekadres, XX, Gemeente, Amsterdam, Pieter,...",298,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Bezoekadres XX Gemeente Amsterdam Pieter Calan...,train,train,"[▁Be, zo, ek, ad, res, ▁XX, ▁Geme, ente, ▁Amst..."


In [42]:
for i in chat_dataset['train_sft'][0]:
    print(i,'\n', chat_dataset['train_sft'][0][i])

prompt 
 Write an informative and persuasive article of 1000 words or more that discusses the environmental and economic benefits of using renewable energy sources, such as solar, wind and hydropower, for powering mining operations. Your article should highlight various case studies, statistics and research findings that demonstrate the feasibility, efficiency and cost-effectiveness of renewable energy solutions in the mining industry. Also, provide insight into the potential challenges, barriers and solutions for implementing renewable energy projects in mining sites worldwide. Use credible sources, technical terms and examples to strengthen your arguments and engage your readers. Finally, offer practical recommendations, policy suggestions or call to actions that inspire readers to support and promote the transition towards a cleaner and more sustainable mining industry.
prompt_id 
 d89a653d6d671c30ef07c318e46ef44df29f25079b2dd5d21ade1af4db7efe0a
messages 
 [{'content': 'Write an inf

In [64]:
def format_message(input_txt, label):
    message_user = {
        "content":input_txt,
        'role':'user'
    }

    message_model = {
        "content":label,
        'role':'assistant'
    }

    return [message_user, message_model]



def format_data(df, text_col, model_token_col, label_col, split_col,  token_threshold='full_text'):
    # df['2split'] = ['dev', 'val']
    display(df)

    format_df = pd.DataFrame(columns=['prompt_id', 'message', split_col])

    for index, row in df.iterrows():

        # select whole text
        if token_threshold == 'full_text':
            input_txt = row[text_col]

        # else select text according to the token threshold
        else:
            # select first n (= token_theshold) tokens using the model tokenizer
            tokens = row[model_token_col][0:token_threshold]

            # combine tokens into txt
            tokens_txt = ''.join(tokens)

            # \n is converted by tokenizer to <0x0A>, we reverse this to get original length
            len_char = len(tokens_txt.replace("<0x0A>", "\n")) # get character length

            # select the same amount of characters as the tokens
            input_txt = row[text_col][0:len_char]

        
        # format message
        label = row[label_col]
        message = format_message(input_txt, label)

        # save in dataframe
        format_df.loc[len(format_df)] = {'prompt_id':row['id'], 'message':message, split_col:row[split_col]}

    # split data
    # if split_col = 4split -> split into dev, train, val and test
    # if split_col = 2split -> split into train and test. Dev and val will be left empty
    train_set = format_df.loc[format_df[split_col]=='train'].drop(columns=[split_col])
    test_set = format_df.loc[format_df[split_col]=='test'].drop(columns=[split_col])
    dev_set = format_df.loc[format_df[split_col]=='val'].drop(columns=[split_col])
    val_set = format_df.loc[format_df[split_col]=='dev'].drop(columns=[split_col])
    display(train_set)
    # chat_dataset = DatasetDict({
    #     'train':train_

    # })
     

            


format_data(tokens_df, 'text', 'GEITjeTokens', 'label', '2split', 50)

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,4split,2split,GEITjeTokens
12190,Raadsadres,/home/azureuser/cloudfiles/code/blobfuse/raads...,24894,test,x Gemeente\n\n% Amsterdam\n\n1 Amsterdam.nl\n\...,"[x, Gemeente, %, Amsterdam, 1, Amsterdam.nl, 1...",55,"[Gemeente, Amsterdam, Amsterdam.nl, 1.1, Formu...",31,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Amsterdam.nl 1.1 Formulier ...,train,train,"[▁x, ▁Geme, ente, <0x0A>, <0x0A>, %, ▁Amsterda..."
7210,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,13829,test,Bezoekadres XX Gemeente Amsterdam\nPieter Cala...,"[Bezoekadres, XX, Gemeente, Amsterdam, Pieter,...",515,"[Bezoekadres, XX, Gemeente, Amsterdam, Pieter,...",298,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Bezoekadres XX Gemeente Amsterdam Pieter Calan...,train,train,"[▁Be, zo, ek, ad, res, ▁XX, ▁Geme, ente, ▁Amst..."


,prompt_id,message
0,24894,[{'content': 'x Gemeente % Amsterdam 1 Amste...
1,13829,[{'content': 'Bezoekadres XX Gemeente Amsterda...
